#  Online Retail Orders with Akka and Db2 Event Store

This notebook uses the [Scala](https://www.scala-lang.org/) programming language
to interact with IBM Db2 Event Stream. It demonstrates how to:

* Connect to Event Store
* Show the data that you inserted using WebSockets, Alpakka, Akka Streams and Db2 Event Store
* Manipulate and aggregate the data with Spark SQL
* Visualize the information with interactive Brunel charts

Run this notebook after running the setup notebook and after (or while)
feeding data in via the `db2-event-store-akka-streams` Scala app.

## Connect to IBM Db2 Event Store

### Determine the IP address of your host

Obtain the IP address of the host that you want to connect to by running the appropriate command for your operating system:

* On Mac, run: `ifconfig`
* On Windows, run: `ipconfig`
* On Linux, run: `hostname -i`

Edit the `host = "XXX.XXX.XXX.XXX"` value in the next cell to provide the IP address.

In [156]:
// Set your host IP address
var host = "192.168.0.105"

// Port will be 1100 for version 1.1.2 or later (5555 for version 1.1.1)
var port = "1100"

// Table name
var tableName = "OnlineRetailOrderDetail"

// Cancellations table name
var cancelTableName = "OnlineRetailCancelDetail"

// Database name
var db = "TESTDB"

## Add Brunel integration
Use cell magic to install the Brunel integration for Apache Toree (Scala).

In [157]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


## Import Scala packages

Import packages for Scala, Spark, and IBM Db2 Event Store.

In [158]:
import sys.process._
import java.io.File
import scala.concurrent.{Await, Future}
import scala.concurrent.duration.Duration
import org.apache.log4j.{Level, LogManager, Logger}
import org.apache.spark._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.ibm.event.EventSession
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import com.ibm.event.catalog.TableSchema
import com.ibm.event.common.ConfigurationReader
import com.ibm.event.example.DataGenerator
import com.ibm.event.oltp.EventContext
import com.ibm.event.oltp.InsertResult

## Connect to Event Store

In [159]:
ConfigurationReader.setConnectionEndpoints(host + ":" + port)
ConfigurationReader.setConnectionTimeout(1)

## Load data from the Event Store table into a DataFrame

In [160]:
val sqlContext = new EventSession(spark.sparkContext, db)
import sqlContext.implicits._

var table = sqlContext.loadEventTable(tableName)
table.registerTempTable(tableName)
var cancelTable = sqlContext.loadEventTable(cancelTableName)
cancelTable.registerTempTable(cancelTableName)

## Show the count and latest rows
Running this cell over and over is the fastest way to see that you have
successfully been inserting rows.

> Note: A timestamp was used for the `id`, so we can use that to sort events as they were received.

In [161]:
sqlContext.sql(s"select count(*) from $tableName").show()
sqlContext.sql(s"select count(*) from $cancelTableName").show()

+--------+
|count(1)|
+--------+
|    2944|
+--------+

+--------+
|count(1)|
+--------+
|      27|
+--------+



In [162]:
sqlContext.sql(s"select * from $tableName order by id desc").show(3)  // Show most recent 3

+-------------+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|           id|InvoiceNo|StockCode|         Description|Quantity|         InvoiceDate|UnitPrice|CustomerId|       Country|
+-------------+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|1542057370269|   536592|    84923|PINK BUTTERFLY HA...|       1|2010-12-02 01:06:...|     4.21|          |United Kingdom|
|1542057370253|   536592|    84915|HAND TOWEL PINK F...|       2|2010-12-02 01:06:...|     4.21|          |United Kingdom|
|1542057370242|   536592|   84913B|MINT GREEN ROSE T...|       2|2010-12-02 01:06:...|     3.36|          |United Kingdom|
+-------------+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
only showing top 3 rows



Cell magic provides a is a prettier way of showing the latest 3 rows:

In [163]:
%%dataframe --limit 3
sqlContext.sql(s"select * from $tableName order by id desc")

id,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
1542057370269,536592,84923,PINK BUTTERFLY HANDBAG W BOBBLES,1,2010-12-02 01:06:00.0,4.21,,United Kingdom
1542057370253,536592,84915,HAND TOWEL PINK FLOWER AND DAISY,2,2010-12-02 01:06:00.0,4.21,,United Kingdom
1542057370242,536592,84913B,MINT GREEN ROSE TOWEL,2,2010-12-02 01:06:00.0,3.36,,United Kingdom


## Aggregate and show the data

Use Spark SQL to build DataFrames with aggregated data.

### Calculate aggregated  gross sales and units by product
#### Show top 10 by gross sales

In [164]:
%%dataframe --limit 10
sqlContext.sql("select Description as Product, sum(Quantity) as Units, sum(CAST (UnitPrice as DECIMAL(15,2)) * Quantity) as Gross" + s" from $tableName" + " group by Description order by 3 desc")

Product,Units,Gross
RED WOOLLY HOTTIE WHITE HEART.,551,1670.55
BLACK RECORD COVER FRAME,480,1627.20
REGENCY CAKESTAND 3 TIER,115,1434.84
WHITE HANGING HEART T-LIGHT HOLDER,439,1153.29
CHILLI LIGHTS,275,1109.55
JUMBO BAG RED RETROSPOT,555,934.51
PAPER CHAIN KIT 50'S CHRISTMAS,273,888.15
SET OF 3 COLOURED FLYING DUCKS,174,813.90
FAIRY TALE COTTAGE NIGHTLIGHT,462,684.90
WOODEN OWLS LIGHT GARLAND,200,681.04


#### Show top 10 by units

In [165]:
%%dataframe --limit 10
val aggDF = sqlContext.sql("select Description as Product, sum(Quantity) as Units, CAST (sum(UnitPrice * Quantity) as DECIMAL(15,0)) as Gross" + s" from $tableName" + " group by Description order by 2 desc")


Product,Units,Gross
NAMASTE SWAGAT INCENSE,600,144
JUMBO BAG RED RETROSPOT,555,935
RED WOOLLY HOTTIE WHITE HEART.,551,1671
STRAWBERRY CERAMIC TRINKET BOX,549,657
RED TOADSTOOL LED NIGHT LIGHT,483,634
BLACK RECORD COVER FRAME,480,1627
FAIRY TALE COTTAGE NIGHTLIGHT,462,685
WHITE HANGING HEART T-LIGHT HOLDER,439,1153
DISCO BALL CHRISTMAS DECORATION,344,38
LADS ONLY TISSUE BOX,338,425


## Visualize the data with Brunel

### Gross sales by product

In [166]:
%%brunel data('aggDF') 
bar at(0,0,50,100) title("Gross Sales by Product")
  x(Product) y(Gross) top(Gross:15)
  tooltip(Product, Gross) color(Gross:BlueGreens) legends(none) opacity(#selection:.5)
  axes(x:'Top 15 Products',y:'Gross') sort(Gross) interaction(select)|
treemap at(55,5,100,70)
  size(Gross) top(Gross:1000) sort(Gross)
  color(Gross:BlueGreens) label(Gross) legends(none)
  tooltip(Product, Gross, Units) opacity(#selection:.5) interaction(select)
:: width=1000, height=600

In [171]:
%%brunel data('aggDF') 
bar at(0,0,50,100) title("Units by Product")
  x(Product) y(Units) top(Units:15)
  tooltip(Product, Units) color(Units:PurpleBlues) legends(none) opacity(#selection:.5)
  axes(x:'Top 15 Products',y:'Units') sort(Units) interaction(select)|
treemap at(55,5,100,70)
  sort(Units) size(Units)  top(Units:1000)
  color(Units:PurpleBlues) label(Units) legends(none)
  tooltip(Product, Gross, Units) opacity(#selection:.5) interaction(select)
:: width=1000, height=600

## Manipulate the the data

For example, we can take the timestamp and determine year, month, day, day-of-week and hour-of-day.
Let's add some time dimensions to the DataFrame and look at what hour of the day is the busiest.

In [168]:
val invoicesDF = sqlContext.sql(
  "select distinct InvoiceNo, InvoiceDate, Country"
  + s" from $tableName group by InvoiceNo, InvoiceDate, Country"
)
val invoicesHourDF = invoicesDF.withColumn("Hour",date_format(invoicesDF("InvoiceDate"), "H")).drop("InvoiceDate")
invoicesHourDF.show(5)

+---------+--------------+----+
|InvoiceNo|       Country|Hour|
+---------+--------------+----+
|   536590|United Kingdom|   0|
|   536425|United Kingdom|  20|
|   536568|United Kingdom|  23|
|   536403|   Netherlands|  19|
|   536576|United Kingdom|   0|
+---------+--------------+----+
only showing top 5 rows



### Invoices by country

In [169]:
%%brunel data('invoicesHourDF')
x(Country) y(#count) sort(#count) color(#count:Reds) at(0,0,50,100) opacity(#selection:.7)
stack bar
  axes(x:'Country', y:'Invoices')
  interaction(select) tooltip(#all)|
map('Europe') opacity(#selection:.7) at(50,0,100,100) x(Country) color(#count:Reds) tooltip(#all)
:: width=1000, height=300


### Invoices by hour and country

In [170]:
%%brunel data('invoicesHourDF') 
x(Hour:linear) y(#count)
stack bar
  axes(x:'Hour':linear, y:'Invoices')
  color(Country:[PurpleBlues,GreenYellows])
:: width=1000, height=200

<p><font size=-1 color=gray>
&copy; Copyright 2018 IBM Corp. All Rights Reserved.
<p>
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.
</font></p>